In [146]:
import pandas as pd
import numpy as np

In [147]:
reviews_detailed = pd.read_csv("reviews_detailed.csv")
calendar_detailed = pd.read_csv("calendar_detailed.csv")
listings_detailed = pd.read_csv("listings_detailed.csv")
# neighbourhoods = pd.read_csv("neighbourhoods.csv")

In [148]:
print(reviews_detailed)

                listing_id                  id        date  reviewer_id  \
0                     3781            37776825  2015-07-10     36059247   
1                     3781            41842494  2015-08-09     10459388   
2                     3781            45282151  2015-09-01     12264652   
3                     3781            49022647  2015-09-30     41426327   
4                     3781            52503327  2015-10-30     15151513   
...                    ...                 ...         ...          ...   
174507  893975999133947853  898547534751030603  2023-05-24    516208792   
174508  893975999133947853  908681741437409696  2023-06-07    169155474   
174509  896170185862100440  918129405470822974  2023-06-20     14805192   
174510  896426215566639130  914465954641543171  2023-06-15     23667572   
174511  896426215566639130  917361890848557786  2023-06-19    202783394   

       reviewer_name                                           comments  
0               Greg  The

In [142]:
listings_detailed.drop(["scrape_id","last_scraped","source"], axis = 1, inplace = True)

In [145]:
listings_detailed.columns

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
  

In [115]:
listings_detailed.columns

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
  

In [116]:
property_reviews = listings_detailed[["id","name",'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value']]

listings_detailed.drop(['number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value'], axis = 1, inplace = True)

In [117]:
property_review.columns

Index(['id', 'name', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value'],
      dtype='object')

In [118]:
listings_detailed.columns

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
  

In [119]:
host_details = listings_detailed[['id', 'name', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url','host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms']]

listings_detailed.drop(['host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url','host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'], axis = 1, inplace = True)

In [133]:
host_details.columns

Index(['id', 'name', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms'],
      dtype='object')

In [137]:
host_details[["host_listings_count","host_total_listings_count"]]

,host_listings_count,host_total_listings_count
0,4730,5621
1,20,42
2,82,132
3,1,1
4,299,1491
...,...,...
3968,299,1491
3969,1,1
3970,4730,5621
3971,4730,5621


In [140]:
listings_detailed.columns

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'license', 'instant_bookable', 'reviews_per_month'],
      dtype='object')

In [127]:
neighbourhood = listings_detailed[["id","name","neighbourhood_cleansed"]]
listings_detailed.drop(['neighbourhood', 'neighbourhood_cleansed','neighbourhood_group_cleansed'], axis=1, inplace = True)

In [132]:
neighbourhood.columns

Index(['id', 'name', 'neighbourhood_cleansed'], dtype='object')

In [130]:
listings_detailed.columns

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'license', 'instant_bookable', 'reviews_per_month'],
      dtype='object')

In [97]:
for location in listings_detailed["neighbourhood"].unique():
    

Boston
nan
Brighton, Massachusetts, United States
Everett, Massachusetts, United States
East Boston, Massachusetts, United States
Boston, Allston, Ma, United States
Charlestown, Massachusetts, United States
Boston (Jamaica Plain), Massachusetts, United States
Roslindale, Massachusetts, United States
BOSTON, Massachusetts, United States
Brookline, Massachusetts, United States
Somerville, Massachusetts, United States
Jamaica Plain , Massachusetts, United States
Boston , Massachusetts, United States
Dorchester, Massachusetts, United States
Jamaica Plain, Massachusetts, United States
West Roxbury, Massachusetts, United States
Allston, Massachusetts, United States
Milton, Massachusetts, United States
Hyde Park, Massachusetts, United States
Hyde Park , Massachusetts, United States
波士顿, Massachusetts, United States
Dedham, Massachusetts, United States
Dorchster , Massachusetts, United States
Dorchester , Massachusetts, United States
Boston, , Massachusetts, United States
Jamaica Plain, Boston

In [93]:
listings_detailed = listings_detailed.replace('Boston, Massachusetts, United States', 'Boston')

In [111]:
listings_detailed["neighbourhood_cleansed"].unique()

array(['South Boston Waterfront', 'North End', 'South Boston',
       'East Boston', 'Allston', 'Downtown', 'Roxbury', 'Roslindale',
       'Mattapan', 'Charlestown', 'Brighton', 'Dorchester', 'Fenway',
       'Bay Village', 'South End', 'Back Bay', 'Beacon Hill',
       'Jamaica Plain', 'Chinatown', 'Mission Hill', 'West End',
       'West Roxbury', 'Hyde Park', 'Longwood Medical Area',
       'Leather District'], dtype=object)

In [80]:
neighbourhoods.drop(["neighbourhood_group"], axis = 1,inplace = True)
neighbourhoods

,neighbourhood
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton
5,Charlestown
6,Chinatown
7,Dorchester
8,Downtown
9,East Boston


In [110]:
len(neighbourhoods)

26

In [112]:
for i in neighbourhoods["neighbourhood"]:
    if i not in listings_detailed["neighbourhood_cleansed"].unique():
        print(i)

Harbor Islands
